In [2]:
! sudo cp /home/jovyan/work/jars/neo4j-connector-apache-spark_2.12-4.1.0_for_spark_3.jar /usr/local/spark/jars/neo4j-connector-apache-spark_2.12-4.1.0_for_spark_3.jar

In [3]:
import pyspark
from pyspark.sql import SparkSession
# NEO4J  CONFIGURATION
bolt_url = "bolt://neo4j:7687"
# Spark init
spark = SparkSession.builder \
    .master("local") \
    .appName('jupyter-pyspark') \
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

23/04/05 17:15:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
#5 
cipher_ql = '''
Match (s:Supplier)-[:SUPPLIES]->(p:Product)
where p.discontinued=false and s.country="USA" 
return  s.companyName,s.contactName, s.country, p.productName, p.unitPrice, p.unitsInStock, p.discontinued
'''
#RETURN c.code, c.level, t.semester, f.name, f.title
df2 = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", bolt_url) \
  .option("query",cipher_ql) \
  .load()
df2.toPandas()

,s.companyName,s.contactName,s.country,p.productName,p.unitPrice,p.unitsInStock,p.discontinued
0,New Orleans Cajun Delights,Shelley Burke,USA,Louisiana Hot Spiced Okra,17.00,4,False
1,New Orleans Cajun Delights,Shelley Burke,USA,Chef Anton's Cajun Seasoning,22.00,53,False
2,New Orleans Cajun Delights,Shelley Burke,USA,Louisiana Fiery Hot Pepper Sauce,21.05,76,False
3,New Orleans Cajun Delights,Shelley Burke,USA,Chef Anton's Cajun Seasoning,22.00,53,False
4,New Orleans Cajun Delights,Shelley Burke,USA,Louisiana Fiery Hot Pepper Sauce,21.05,76,False
...,...,...,...,...,...,...,...
72,New England Seafood Cannery,Robb Merchant,USA,Boston Crab Meat,18.40,123,False
73,New England Seafood Cannery,Robb Merchant,USA,Jack's New England Clam Chowder,9.65,85,False
74,New England Seafood Cannery,Robb Merchant,USA,Boston Crab Meat,18.40,123,False
75,New England Seafood Cannery,Robb Merchant,USA,Boston Crab Meat,18.40,123,False


In [6]:
#6
from pyspark.sql.functions import col, concat, lit
e = spark.read.option("multiline",True) \
    .json("file:///home/jovyan/datasets/fudgemart/fudgemart-employees.json") \
    .withColumn("employee_name", concat(col("employee_firstname"),lit(" "),col("employee_lastname")) )
e.toPandas()


,employee_birthdate,employee_department,employee_firstname,employee_fulltime,employee_hiredate,employee_hourlywage,employee_id,employee_jobtitle,employee_lastname,employee_ssn,employee_supervisor_id,employee_termdate,employee_name
0,1982-01-12T00:00:00,Electronics,Arial,0,2011-07-05T00:00:00,15.2830,1,Sales Associate,Photo,111220001,5.0,None,Arial Photo
1,1982-11-30T00:00:00,Electronics,Sal,1,2005-07-26T00:00:00,15.0699,2,Sales Associate,Ladd,111220002,5.0,None,Sal Ladd
2,1972-09-03T00:00:00,Hardware,Dustin,0,2004-07-02T00:00:00,12.4500,3,Sales Associate,Dawind,111220003,6.0,2010-11-06T00:00:00,Dustin Dawind
3,1990-05-13T00:00:00,Hardware,Sandi,1,2011-01-02T00:00:00,13.0997,4,Sales Associate,Shores,111220004,6.0,None,Sandi Shores
4,1974-02-22T00:00:00,Electronics,Isabelle,1,2005-08-16T00:00:00,20.1287,5,Department Manager,Gunnering,111220005,32.0,None,Isabelle Gunnering
5,1973-07-29T00:00:00,Hardware,Lee,1,2004-01-26T00:00:00,18.5845,6,Department Manager,Hvmeehom,111220006,32.0,None,Lee Hvmeehom
6,1988-04-03T00:00:00,Housewares,Allan,1,2005-02-22T00:00:00,12.0878,7,Sales Associate,Wrench,111220007,24.0,None,Allan Wrench
7,1973-12-13T00:00:00,Sporting Goods,Ally,1,2005-08-14T00:00:00,12.3541,8,Sales Associate,Gator,111220008,27.0,None,Ally Gator
8,1980-11-30T00:00:00,Housewares,Alma,1,2004-11-12T00:00:00,11.7152,9,Sales Associate,Frienzergon,111220009,24.0,None,Alma Frienzergon
9,1976-10-02T00:00:00,Hardware,Artie,1,2005-04-04T00:00:00,12.7269,10,Sales Associate,Choke,111220010,6.0,None,Artie Choke


In [10]:
from pyspark.sql.functions import concat_ws
e.select( concat_ws("-", "employee_jobtitle", "employee_department").alias("foo") ).toPandas()

,foo
0,Sales Associate-Electronics
1,Sales Associate-Electronics
2,Sales Associate-Hardware
3,Sales Associate-Hardware
4,Department Manager-Electronics
5,Department Manager-Hardware
6,Sales Associate-Housewares
7,Sales Associate-Sporting Goods
8,Sales Associate-Housewares
9,Sales Associate-Hardware


In [7]:
emps = .select("employee_name","employee_department", "employee_jobtitle","employee_id").distinct()

SyntaxError: invalid syntax (837298648.py, line 1)

In [12]:
#7
cipher_ql = '''
MERGE (f:Employee {
    name: event.employee_name , 
    department: event.employee_department, 
    jobtitle: event.employee_jobtitle, 
    id: event.employee_id 
})
'''
emps = e.select("employee_name","employee_department", "employee_jobtitle","employee_id").distinct()
emps.write.format("org.neo4j.spark.DataSource").mode("Overwrite") \
  .option("url", bolt_url) \
  .option("query",cipher_ql) \
  .save()


In [9]:
equery = "match (e:Employee) RETURN e.department;"
demo = spark.read.format("org.neo4j.spark.DataSource") \
    .option("url", bolt_url) \
    .option("query",equery) \
    .load()

demo.show()

+----------------+
|    e.department|
+----------------+
|        Hardware|
|  Sporting Goods|
|     Electronics|
|        Clothing|
|      Housewares|
|  Sporting Goods|
|Customer Service|
|      Housewares|
|Customer Service|
|     Electronics|
|        Hardware|
|     Electronics|
|        Hardware|
|Customer Service|
|        Clothing|
|        Clothing|
|        Hardware|
|      Housewares|
|  Sporting Goods|
|        Hardware|
+----------------+
only showing top 20 rows



In [35]:
demo.select("e.<id>").show()

+----+
|<id>|
+----+
|  18|
|  19|
|  20|
|  21|
| 138|
| 139|
| 140|
| 141|
| 142|
| 143|
| 144|
| 145|
| 146|
| 147|
| 148|
| 149|
| 150|
| 151|
| 152|
| 153|
+----+
only showing top 20 rows



In [15]:
e.select("employee_id","employee_name","employee_supervisor_id").show()

+-----------+------------------+----------------------+
|employee_id|     employee_name|employee_supervisor_id|
+-----------+------------------+----------------------+
|          1|       Arial Photo|                     5|
|          2|          Sal Ladd|                     5|
|          3|     Dustin Dawind|                     6|
|          4|      Sandi Shores|                     6|
|          5|Isabelle Gunnering|                    32|
|          6|      Lee Hvmeehom|                    32|
|          7|      Allan Wrench|                    24|
|          8|        Ally Gator|                    27|
|          9|  Alma Frienzergon|                    24|
|         10|       Artie Choke|                     6|
|         11|       Bette Alott|                    27|
|         12|      Bill Melator|                    27|
|         13|       Bob Enweave|                    27|
|         14|   Chris P. Nugget|                     5|
|         15|       Chuck Itupp|                

In [41]:
e.select("employee_id","employee_name","employee_supervisor_id").orderBy("employee_supervisor_id").toPandas()

,employee_id,employee_name,employee_supervisor_id
0,33,Mike Fudge,NaN
1,1,Arial Photo,5.0
2,2,Sal Ladd,5.0
3,14,Chris P. Nugget,5.0
4,21,Maxi Mumm,5.0
5,3,Dustin Dawind,6.0
6,4,Sandi Shores,6.0
7,10,Artie Choke,6.0
8,16,Detyers Erin,6.0
9,17,Kurt Tan,20.0


In [14]:
#8 add Supervisor relationships
cipher_ql = '''
MATCH (e:Employee)
MATCH (s:Employee)
WHERE e.id = event.employee_id AND 
    s.id = event.employee_supervisor_id
MERGE (s)-[:SUPERVISES]->(e)
'''
e.select("employee_id","employee_name","employee_supervisor_id").write.format("org.neo4j.spark.DataSource").mode("Overwrite") \
  .option("url", bolt_url) \
  .option("query",cipher_ql) \
  .save()


In [15]:
!pip install networkx